<a href="https://colab.research.google.com/github/olaviinha/MusicWithChatGPT/blob/main/chords2midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">chords2mid<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Music with ChatGPT</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/MusicWithChatGPT" target="_blank"><font color="#999" size="4">Github</font></a>

Saves chord progressions as midi.

---

- Run the _Setup_ cell by pressing the play button right under the _Setup_ title.
- A box will open on the right side of the screen. You may copy-paste a chord progression directly from ChatGPT to that box and run the _Download chord progression as .mid_ cell to immediately download whatever you currently have in the box, as a MIDI file. You can paste new chord progression in the box and run the cell again, as often and many times as you like.
- All valid chords will be parsed from the text entered in the right side box. Any lines of descriptive text that mention chord(s) should be removed, as otherwise those chords will also end up in the MIDI file.
- `humanize` will add light randomized variation to note velocities (volume) and start times.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will setup prerequisites.<br>
# #@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
repositories = []
pip_packages = 'pychord mido'
apt_packages = ''
mount_drive = False #@ param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output, files
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys
from datetime import timedelta
import math
from pychord import Chord
import librosa
from mido import Message, MidiFile, MidiTrack, MetaMessage, bpm2tempo

chords_file = "/content/Paste chord progression here"
!echo '' > "{chords_file}"

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.', time=True)


files.view(chords_file)
files.view(chords_file)

In [ ]:
#@title ## Download chord progression as `.mid`
octave = 3 #@param {type:"integer"}
bpm = 120 #@param {type:"integer"}
humanize = True #@param {type:"boolean"}
repeat_progression = 1 #@ param {type:"integer"}


chords_per_bar = 1
uniq_id = gen_id()
timer_start = time.time()

with open(chords_file, 'r') as f:
  chords_string = f.read()

words = ' '.join(chords_string.split('\n')).split(' ')

chords = []
for word in words:
  try:
    Chord(word)
    chords.append(word)
  except:
    pass

chords_found = len(chords)
midi_chords = []
for chord in chords:
  chord = Chord(chord)
  notes = chord.components_with_pitch(root_pitch=octave)
  midi_chords.append([librosa.note_to_midi(note) for note in notes])

op(c.title, 'Generating MIDI from '+str(chords_found)+' chords:')
print(' '.join(chords))

octave_shift = 0
velocity_min = 64
velocity_max = 84

tpb = 128
bar = tpb
humanization_amount = 9

mid = MidiFile(ticks_per_beat=tpb)
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
track.append(MetaMessage('set_tempo', tempo=bpm2tempo(bpm), time=0))

for repeat in range(repeat_progression):
  for i, chord in enumerate(midi_chords, 1):
    velocity = random.randrange(velocity_min, velocity_max) if velocity_min != velocity_max else velocity_max
    start = 0 if humanize == None else random.randint(0, humanization_amount)
    end = bar
    for note in chord:
      track.append(Message('note_on', note=int(note), velocity=velocity, time=start))
    for note in chord:
      track.append(Message('note_off', note=int(note), velocity=tpb-1, time=end))

file_out = '/content/'+uniq_id+'.mid'
mid.save(file_out)

print()
if os.path.isfile(file_out):
  op(c.ok, 'Downloading...')
  files.download(file_out)
else:
  op(c.fail, 'Error occurred.')

timer_end = time.time()

print()
# op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
# op(c.ok, 'FIN.')

# if end_session_when_done is True: end_session()